# Chapter 1. Introduction to functional programming

## 1.1 What is this thing called functional programming?

At a very high level, it is a programming style that emphasises on
1. Functions as first-class values
2. Avoiding state mutation

### 1.1.1 Functions as first-class values

* allows function to be used as input and output of other functions
* assign function to variables - in other words, you can treat it like a value

treating function as value allow us to write very concise and powerful code

In [1]:
Func<int, int> triple = x => x * 3;
var range = Enumerable.Range(1, 3);
var triples = range.Select(triple);
triples

index,value
0,3
1,6
2,9


### 1.1.2 Avoiding state mutation

When following functional paradigm, we should refrain from state mutation.

* once created, an object `never` changes and variables should `never` be reassigned

`muataion` means that a value is changed in-place. It is also referred to as destructive updates


In [1]:
var nums = new List<int>{1, 2, 3 };
nums[2] = 7;

nums // original list is mutated

index,value
0,1
1,2
2,7


In [1]:
// function / operation that does mutaion
var nums = new List<int>{3, 2, 1};
nums.Sort();
nums


index,value
0,1
1,2
2,3


In [1]:

// functional version - no mutation
var nums1 = new List<int>{3, 2, 1};
var result = nums1.OrderBy(x => x);
(nums1, result)

Item1,Item2
"[ 3, 2, 1 ]","[ 1, 2, 3 ]"


### 1.1.3 Writing programs with strong grarantees



In [1]:
// Example of why state mutation is bad

using static System.Linq.Enumerable;
using static System.Console;

var list = Range(-10000, 20001).ToList();
Action task1 = () => {WriteLine(list.Sum());};
Action task2 = () => {list.Sort(); WriteLine(list.Sum());};
Parallel.Invoke(task1, task2);
// expect 0 from both tasks as result but due to in-place mutation, we get wrong result


37


0


In [1]:

// Example without state mutation

using static System.Linq.Enumerable;
using static System.Console;

var list = Range(-10000, 20001).ToList();
Action task1 = () => {WriteLine(list.Sum());};
Action task2 = () => {list.OrderBy( x => x); WriteLine(list.Sum());};
Parallel.Invoke(task1, task2);

0


0


> "Although most people will agree that treating functions as first-class values and avoiding state mutation are fundamental tenets of FP, their application gives rise to a series of practices and techniques, so it’s debatable which techniques should be considered essential and included in a book like this. I encourage you to take a pragmatic approach to the subject and try to understand FP as a set of tools that you can use to address your programming tasks. As you learn these techniques, you’ll start to look at problems from a different perspective: you’ll start to think functionally"

#### Functional vs. object-oriented?

Fundamental priciples of OOP (encapsuation, data abstration, and so on) are `orthogonal` to the priciples of FP, so there's no reason why the two paradigms can't be combined.

The real question is `imperative vs functional`

The idea of structuring a large and complex appication relies on serveral principles
* Modularity
* Separation of concerns
* Layering
* Loose coupling

These can be acheived in both OOP and FP - the mechanics of acheiving them might be different

## 1.2 How functional a language is C#?